<a href="https://colab.research.google.com/github/machine-perception-robotics-group/MPRGDeepLearningLectureNotebook/blob/master/12_gan/15_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Encoderによる画像の復元とデノイジング


---
## 目的

Auto Encoder (AE)を用いることで，画像の復元とデノイジング (ノイズの除去)を行う．
今回は， MNIST Datasetに対して人為的にスパイクノイズを付与し，画像の復元とデノイジングを行う．

## 準備

### Google Colaboratoryの設定確認・変更
本チュートリアルではPyTorchを利用してニューラルネットワークの実装を確認，学習および評価を行います．
**GPUを用いて処理を行うために，上部のメニューバーの「ランタイム」→「ランタイムのタイプを変更」からハードウェアアクセラレータをGPUにしてください．**


## モジュールのインポート
はじめに必要なモジュールをインポートする．

### GPUの確認
GPUを使用した計算が可能かどうかを確認します．

`GPU availability: True`と表示されれば，GPUを使用した計算を行うことが可能です．
Falseとなっている場合は，上記の「Google Colaboratoryの設定確認・変更」に記載している手順にしたがって，設定を変更した後に，モジュールのインポートから始めてください．

In [ ]:
# モジュールのインポート
import os
from time import time
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms

# GPUの確認
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

## データセットの作成

PyTorchのデータセットオブジェクトを作成します．
今回はtorchvisionが提供している`MNIST`データセットのオブジェクトをベースにして，作成を行います．

`class DenoiseMNIST(torchvision.datasets.MNIST)`にて，元のMNISTを親クラスとして継承します．

そして，`__init__`関数では，引数はMNISTと同様のものを定義します．
ここで，`super(DenoiseMNIST, self).__init__(root, train, transform, target_transform, download)`とすることで，元のMNISTクラスが行う`__init__`関数の処理を実行します．
これにより，MNISTクラスが保有するデータ（self.data, self.targets）を同じように読み込み，`DenoiseMNIST`内に保存することができます．

`__len__`関数に関しては，元のMNISTデータセットと同様のものを使用するため，記述を行いません．

次に，`__getitem__`関数を定義します．
まず，指定した`index`のデータを`self.data, self.targets`から選択します．ただし，分類問題の正解ラベルである`target`は使用しないことに注意してください．
`self.data`より選択された画像データは，ノイズを含んでいないデータのため，`denoised`（正解ラベル）として変数へ保存します．
次に，画像にスパイクノイズを付与します．
スパイクノイズは乱数のマスクを作成し，学習サンプルとマスクの値を掛け合わせることでノイズを付与します．

そして，ノイズを含んだ画像（`img`）およびノイズを含まない画像（`denoised`）を返します．

In [ ]:
class DenoiseMNIST(torchvision.datasets.MNIST):

    def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
        ### 通常のMNISTデータセットの__init__を実行（データのダウンロード・読み込み）
        super(DenoiseMNIST, self).__init__(root, train, transform, target_transform, download)

        self.noise_ratio = 0.25  # ノイズ比の設定

    def __getitem__(self, index):
        ### 画像データの読み込み（ラベルは使用しない）
        denoised = self.data[index]

        ### ノイズ画像の作成
        # ランダムノイズの生成
        mask = np.random.binomial(size=denoised.size(), n=1, p=1.0-self.noise_ratio)
        mask = torch.from_numpy(mask.astype(np.uint8))
        # ノイズの付与
        noise = denoised
        noise = noise * mask

        ### 前処理（PyTorchのTensor配列への変換など）
        if self.transform is not None:
            noise = Image.fromarray(noise.numpy(), mode='L')
            noise = self.transform(noise)
            denoised = Image.fromarray(denoised.numpy(), mode='L')
            denoised = self.transform(denoised)

        return noise, denoised

### データセットの表示
ノイズを付与する前後の画像を表示してみます．
ここでは，matplotlibを用いて複数の画像を表示させるプログラムを利用します．

In [ ]:
train_data = DenoiseMNIST(root="./", train=True, transform=transforms.ToTensor(), download=True)
test_data = DenoiseMNIST(root="./", train=False, transform=transforms.ToTensor(), download=True)

cols = 10

plt.clf()
fig = plt.figure(figsize=(14, 1.4))
for c in range(cols):
    ax = fig.add_subplot(1, cols, c+1)
    ax.imshow(train_data[c][0].reshape(28, 28), cmap=plt.get_cmap('gray'))
    ax.set_axis_off()
plt.show()

plt.clf()
fig = plt.figure(figsize=(14, 1.4))
for c in range(cols):
    ax = fig.add_subplot(1, cols, c+1)
    ax.imshow(train_data[c][1].reshape(28, 28), cmap=plt.get_cmap('gray'))
    ax.set_axis_off()
plt.show()

## ネットワークモデルの定義

Auto Encoderの定義をします．

今回はEncoder, Decoderそれぞれ全結合層1層からなるAutoEncoderを作成します．

In [ ]:
class AutoEncoder(nn.Module):

    def __init__(self, n_inputs, n_hidden):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Linear(n_inputs, n_hidden)
        self.decoder = nn.Linear(n_hidden, n_inputs)
        self.act = nn.Sigmoid()

    def forward(self, x):
        h = self.act(self.encoder(x))
        h = self.decoder(h)
        return h

## ネットワークの作成
上で定義したネットワークを作成します．

入力層のサイズ（input_num）と中間層のサイズ（hidden_num）を定義し，AutoEncoderのネットワークを作成します．

また，最適化手法としてAdamを使用します．

In [ ]:
input_num = 28 * 28
hidden_num = 2

model = AutoEncoder(n_inputs=input_num, n_hidden=hidden_num)
if use_cuda:
    model.cuda()

optimizer = torch.optim.Adam(model.parameters())

## 学習
先ほど定義したデータセットと作成したネットワークを用いて，学習を行います．

1回の誤差を算出するデータ数（ミニバッチサイズ）を100，学習エポック数を10とします．

次にデータローダーを定義します．
データローダーでは，上で読み込んだデータセット（`train_data`）を用いて，for文で指定したミニバッチサイズでデータを読み込むオブジェクトを作成します．
この時，`shuffle=True`と設定することで，読み込むデータを毎回ランダムに指定します．

次に，誤差関数を設定します．
今回は，連続値を出力する回帰問題をあつかうため，`MSELoss`を`criterion`として定義します．

In [ ]:
batch_size = 100
epoch_num = 10

# データローダーの設定
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)

# 誤差関数の設定
criterion = nn.MSELoss()
if use_cuda:
    criterion.cuda()

# ネットワークを学習モードへ変更
model.train()

# 学習の実行
start = time()
for epoch in range(1, epoch_num+1):
    sum_loss = 0.0

    for image, label in train_loader:
        # 画像データを2次元配列 --> 1次元配列へ変換
        image = image.view(image.size()[0], -1)
        label = label.view(label.size()[0], -1)

        if use_cuda:
            image = image.cuda()
            label = label.cuda()

        y = model(image)
        loss = criterion(y, label)

        model.zero_grad()
        loss.backward()
        optimizer.step()

        sum_loss += loss.item()

    print("epoch:{}, mean loss: {}, elapsed time: {}".format(epoch,
                                                             sum_loss / len(train_loader),
                                                             time() - start))

## テスト

学習したネットワークモデルを用いてデノイジングを行います．
このとき，結果を可視化して確認するために，デノイジングを行なった結果を`test_result`へ格納します，また，対応する正解データを`test_gt`へ格納します．

In [ ]:
# データセットの読み込み・データローダーの設定
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)

# ネットワークを評価モードへ変更
model.eval()

test_result = []
test_gt = []

# デノイジングの実行
with torch.no_grad():
    for count, (image, label) in enumerate(test_loader):
        image = image.view(image.size()[0], -1)
        label = label.view(label.size()[0], -1)
        if use_cuda:
            image = image.cuda()

        y = model(image)

        if use_cuda:
            test_result.append(y.data.cpu())
        else:
            test_result.append(y.data)
        test_gt.append(label.data)

        if count == 10:
            break

デノイジングの結果を表示してみます．
表示にはmatplotlibを用います．


In [ ]:
cols = 10

plt.clf()
fig = plt.figure(figsize=(14, 1.4))
for c in range(cols):
    ax = fig.add_subplot(1, cols, c + 1)
    ax.imshow(test_result[c].reshape(28, 28), cmap=plt.get_cmap('gray'))
    ax.set_axis_off()
fig.suptitle("Denoise result")
plt.show()

plt.clf()
fig = plt.figure(figsize=(14, 1.4))
for c in range(cols):
    ax = fig.add_subplot(1, cols, c + 1)
    ax.imshow(test_gt[c][0].reshape(28, 28), cmap=plt.get_cmap('gray'))
    ax.set_axis_off()
fig.suptitle("Ground truth")
plt.show()

# 潜在空間の可視化

学習で獲得した潜在空間の表現を可視化して確認します．
潜在空間の2次元ベクトルを擬似的に作成し，その値をDecoderへと入力し画像を生成することで，潜在空間の値を確認します．



※ この演算は`hidden_num = 2`の場合のAuto Encoderでのみ動作します．

In [ ]:
# 各次元のサンプリング点の生成
nv = 25
minmax_range = 1.0
value1 = np.linspace(0, minmax_range, nv)
value2 = np.linspace(0, minmax_range, nv)

# 結果表示用のNumpy arrayの作成
plot_array = np.zeros([28 * nv, 28 * nv], dtype=np.float32)

# 潜在変数をDecoderへ入力し，画像を生成する
for i, yi in enumerate(value1):
    for j, xj in enumerate(value2):
        xx = torch.tensor([[yi, xj]], dtype=torch.float32)
        xx = xx.cuda()

        with torch.no_grad():
            output = model.decoder(xx)

        output = output.view(-1, 28, 28)
        output = output.cpu().numpy()
        plot_array[(nv-i-1)*28:(nv-i)*28, j*28:(j+1)*28] = output

# 結果の生成
plt.figure(figsize=(10, 15))
Xi, Yi = np.meshgrid(value1, value2)
plt.imshow(plot_array, origin="upper", cmap="gray")
plt.tight_layout()
plt.show()

## 課題

1. 中間層のユニット数や層数を変更した時の性能を比較しましょう．

### ヒント
* Encoder, Decoderそれぞれのユニット数や総数は，対称になるように設計する．